In [1]:
import os
import numpy as np
import shutil
import nibabel as nib
from fnmatch import fnmatch
import re

In [19]:
class brain:

    def __init__(self,path):
        
            #turn the matrix it ones to mark where activation is
        def activationpts(f):
            '''
            function for turning activation points into binary values
            
            f (array/array-like) ; array of matrix of brain activation data
            
            
            '''
            return np.sign(np.abs(f))
    
         #load files through directory so we can link to subj ID
        def get_files(direct):
            
            #load all niis in path
            fels=[os.path.join(direct, k) for k in os.listdir(direct) if re.search('beta.*.nii', k) ]
            
            
            #pull affine and head for zero-eth element in the list
            affine=nib.load(fels[0]).affine
            hdr=nib.load(fels[0]).header
            
            #read them as niis and pull data matrix
            fels=np.array([nib.load(f).get_data() for f in fels])
            
            fels=activationpts(np.sum(fels, axis=0))
            fels[np.isnan(fels)]=0
            return {'data': fels, 'affines': affine, 'header': hdr}
        
        
        
        self.path = path
        
        #dictionary linking subject ID to subject data
        self.data={}
        dirs = os.listdir(path)
        for d in dirs:
            self.data[d]=get_files(os.path.join(path, d))
    
        #merge nii images
        self.data['Group']={'data':'', 'affines': '', 'header': '' }
        self.data['Group']['data']=np.sum(np.array(
            [self.data[sub]['data'] for sub in self.data.keys() if sub !='Group']), axis=0)
        self.data['Group']['affines']=self.data[list(self.data.keys())[0]]['affines']
        self.data['Group']['header']=self.data[list(self.data.keys())[0]]['header']
        
      

    def write_map(self, data='Group', path=None, fileName='Nii1.nii'):
        
        wrt_file=nib.Nifti1Image(self.data[data]['data'], self.data[data]['affines'],
                                 self.data[data]['header'])
        if path:
            wrt_file.to_filename(os.path.join(path, fileName))
        else:
            wrt_file.to_filename(os.path.join(self.path, fileName))
            
   
    def write_mask(self, threshold, data='Group', path=None, fileName='mask.nii'):
        #pull in self.data for readability
        tr_data=self.data[data]['data']
        #zero out sub threshold values then convert to ones
        tr_data[tr_data<threshold]=0
        tr_data=np.sign(tr_data)
        
        wrt_file=nib.Nifti1Image(tr_data, self.data[data]['affines'],
                                 self.data[data]['header'])

        if path:
            wrt_file.to_filename(os.path.join(path, fileName))
        else:
            wrt_file.to_filename(os.path.join(self.path, fileName))
        
        
        
        
        
    
    #finding missing data and pair them with subject ID
    #dirs = os.listdir(self.path)
    #for d in dirs:
        #print (self.data[d])
    def datapt (self,x,y,z):
        names=list(self.data.keys())
        names.remove('Group')
        missinglist = []
        for ID in names: 
            if self.data[ID]['data'][x][y][z] == 0:
                missinglist.append(ID)
            else:
                pass
        return (missinglist)

        
       

In [20]:
k=brain('/mnt/datashare/Pr1.LL/Analysis/Analysis_level1/uniVar_opEv[Final choosen_minus_50dt]')
#k.write_mask(data='Group', threshold=38, path='/mnt/datashare/Pr1.LL/Analysis/', fileName='mask_thresh38.nii')


In [4]:
k.write_mask(data='Group', threshold=1, path='/mnt/datashare/Pr1.LL/Analysis/', fileName='mask_thresh1.nii')